# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [1]:
# TODO: Implement load the data here.
import pickle

training_file = './train.p'
testing_file = './test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [2]:
# TODO: Implement data normalization here.
import numpy as np

#print(X_train[0])
def normalize_data(X):
    X_min = np.min(X)
    X_max = np.max(X)
    X_norm = (X-X_min)/(X_max-X_min)
    return X_norm - 0.5

X_train = normalize_data(X_train)
X_test = normalize_data(X_test)

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [3]:
# TODO: Build a two-layer feedforward neural network with Keras here.
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

n_classes = 43
hidden_layers = 128
image_sizeX = 32
image_sizeY = 32
num_channels = 3 
num_features = image_sizeX * image_sizeY * num_channels

model = Sequential()
model.add(Dense(hidden_layers, input_shape=(num_features,), name="hidden1"))
model.add(Activation('relu'))
model.add(Dense(n_classes, name="output"))

model.add(Activation('softmax'))

model.summary()

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
hidden1 (Dense)                  (None, 128)           393344      dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 128)           0           hidden1[0][0]                    
____________________________________________________________________________________________________
output (Dense)                   (None, 43)            5547        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 43)            0           output[0][0]                     
Total params: 398891
______________________________________________________________________

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [4]:
# TODO: Compile and train the model here.
from keras.optimizers import Adam

def reformat_flat(dataset, labels):
  dataset = dataset.reshape((-1, num_features)).astype(np.float32)
  labels = (np.arange(n_classes) == labels[:,None]).astype(np.float32)
  return dataset, labels

X_train, y_train = reformat_flat(X_train, y_train)
X_test, y_test = reformat_flat(X_test, y_test)

def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
X_train, y_train = randomize(X_train, y_train)
X_test, y_test = randomize(X_test, y_test)

batch_size = 128
nb_epoch = 2

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

Train on 39209 samples, validate on 12630 samples
Epoch 1/2
39209/39209 [==============================] - 8s - loss: 1.6203 - acc: 0.5820 - val_loss: 1.2699 - val_acc: 0.6401
Epoch 2/2
39209/39209 [==============================] - 8s - loss: 0.7645 - acc: 0.8013 - val_loss: 0.9428 - val_acc: 0.7456


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [5]:
# TODO: Split some of the training data into a validation dataset.
from sklearn.cross_validation import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# TODO: Compile and train the model to measure validation accuracy.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)
print(score)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % X_train.shape[0] / float(X_val.shape[0])
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 5s - loss: 0.5592 - acc: 0.8580 - val_loss: 0.5074 - val_acc: 0.8628
Epoch 2/2
29406/29406 [==============================] - 6s - loss: 0.4543 - acc: 0.8822 - val_loss: 0.4218 - val_acc: 0.8999
[0.80539021227724106, 0.79208234364516383]


**Validation Accuracy**: 89.99%

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [9]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
from keras.layers import Convolution2D, Flatten

# Format the data
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']
X_train = normalize_data(X_train)
X_test = normalize_data(X_test)

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_sizeX, image_sizeY, num_channels)).astype(np.float32)
  labels = (np.arange(n_classes) == labels[:,None]).astype(np.float32)
  return dataset, labels

X_train, y_train = reformat(X_train, y_train)
X_test, y_test = reformat(X_test, y_test)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


# Create the network
num_filters = 32
fitler_size = 3
input_shape = (image_sizeX, image_sizeY, num_channels)
model = Sequential()
model.add(Convolution2D(num_filters, fitler_size, fitler_size,
                        border_mode='valid',
                        input_shape=input_shape))
#model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(hidden_layers, input_shape=(28800,), name="hidden1"))
model.add(Activation('relu'))
model.add(Dense(n_classes, name="output"))

model.add(Activation('softmax'))

model.summary()

# TODO: Compile and train the model.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)
print(score)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_3 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 28800)         0           convolution2d_3[0][0]            
____________________________________________________________________________________________________
hidden1 (Dense)                  (None, 128)           3686528     flatten_3[0][0]                  
____________________________________________________________________________________________________
activation_7 (Activation)        (None, 128)           0           hidden1[0][0]                    
___________________________________________________________________________________________

In [ ]:
print(history.history['val_acc'])

**Validation Accuracy**: 93.6%

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [10]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# TODO: Compile and train the model.
from keras.layers import MaxPooling2D

num_filters = 32
fitler_size = 3
pool_size = (2, 2)
input_shape = (image_sizeX, image_sizeY, num_channels)

model = Sequential()
model.add(Convolution2D(num_filters, fitler_size, fitler_size,
                        border_mode='valid',
                        input_shape=input_shape))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(hidden_layers, input_shape=(7200,), name="hidden1"))
model.add(Activation('relu'))
model.add(Dense(n_classes, name="output"))

model.add(Activation('softmax'))

model.summary()

# TODO: Compile and train the model.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)
print(score)


# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_4 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 15, 15, 32)    0           convolution2d_4[0][0]            
____________________________________________________________________________________________________
activation_9 (Activation)        (None, 15, 15, 32)    0           maxpooling2d_1[0][0]             
____________________________________________________________________________________________________
flatten_4 (Flatten)              (None, 7200)          0           activation_9[0][0]               
___________________________________________________________________________________________

AssertionError: The validation accuracy is: 0.808

**Validation Accuracy**: 93.1%

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [11]:
# TODO: Re-construct the network and add dropout after the pooling layer.
# TODO: Compile and train the model.
model = Sequential()
model.add(Convolution2D(num_filters, fitler_size, fitler_size,
                        border_mode='valid',
                        input_shape=input_shape))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.50))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(hidden_layers, input_shape=(7200,), name="hidden1"))
model.add(Activation('relu'))
model.add(Dense(n_classes, name="output"))

model.add(Activation('softmax'))

model.summary()

# TODO: Compile and train the model.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)
print(score)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_5 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_5[0][0]      
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 15, 15, 32)    0           convolution2d_5[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 15, 15, 32)    0           maxpooling2d_2[0][0]             
____________________________________________________________________________________________________
activation_12 (Activation)       (None, 15, 15, 32)    0           dropout_1[0][0]                  
___________________________________________________________________________________________

AssertionError: The validation accuracy is: 0.788

In [ ]:
**Validation Accuracy**: 90.44%

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [13]:
num_filters1 = 32
num_filters2 = 32
fitler_size = 3
pool_size = (2, 2)
hidden_layers = 256
nb_epoch = 5
batch_size = 128

input_shape1 = (image_sizeX, image_sizeY, num_channels)
input_shape2 = (image_sizeX, image_sizeY, num_filters)

model = Sequential()
model.add(Convolution2D(num_filters1, fitler_size, fitler_size,
                        border_mode='valid',
                        input_shape=input_shape1))
model.add(Activation('relu'))
model.add(Convolution2D(num_filters2, fitler_size, fitler_size,
                        border_mode='valid'))
model.add(MaxPooling2D(pool_size=pool_size))
#model.add(Dropout(0.50))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(hidden_layers, input_shape=(7200,), name="hidden1"))
model.add(Activation('relu'))
model.add(Dense(n_classes, name="output"))

model.add(Activation('softmax'))

model.summary()

# TODO: Compile and train the model.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_8 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_7[0][0]      
____________________________________________________________________________________________________
activation_19 (Activation)       (None, 30, 30, 32)    0           convolution2d_8[0][0]            
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 28, 28, 32)    9248        activation_19[0][0]              
____________________________________________________________________________________________________
maxpooling2d_4 (MaxPooling2D)    (None, 14, 14, 32)    0           convolution2d_9[0][0]            
___________________________________________________________________________________________

**Best Validation Accuracy:** 97.55%

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [18]:
from keras.utils import np_utils

print(model.metrics_names)

with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

model.evaluate(X_test, Y_test)

['loss', 'acc']
12630/12630 [==============================] - 14s    


[0.47259636923069059, 0.90467141722273658]

**Test Accuracy:** 90.46%

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.